In [ ]:
import warnings

warnings.filterwarnings('ignore')

import os

import snman
from snman.constants import *
from snman import osmnx_customized as oxc
from snman import *

PERIMETER = 'affoltern'
PROJECT = '_main'

data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','HOSTING','web', 'xxxx snman data package')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PROJECT)
export_path = os.path.join(data_directory, 'outputs', PROJECT)

CRS_internal = 2056      # for Zurich

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [ ]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)

street_graph.organize_edge_directions(G)

In [ ]:
print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    os.path.join(inputs_path, 'rebuilding_regions', 'rebuilding_regions.gpkg'),
    only_active=True,
    crs=CRS_internal
)

if 1:
    print('Load access needs')
    access_needs = snman.io.load_access_needs(
        os.path.join(inputs_path, 'switzerland', 'zurich', 'statpop', 'statpop22_zurich_city_agg_hectars_public.gpkg'),
        CRS_internal
    )
    access_needs['parking_spots_needed'] = access_needs['residents'] / 60
    
if 1:
    print('Load lane edits')
    lane_edits = snman.io.load_lane_edits(
        os.path.join(inputs_path, 'lane_edits', 'lane_edits.gpkg')
    )
    snman.enrichment.match_lane_edits(
        G, lane_edits, lanes_key=KEY_FORCED_GIVEN_LANES_DESCRIPTION,
        max_dist=200, max_dist_init=500, max_lattice_width=5,
    )
    
if 1:
    print('Load hierarchy edits')
    hierarchy_edits = snman.io.load_hierarchy_edits(
        os.path.join(inputs_path, 'hierarchy_edits', 'hierarchy_edits.gpkg')
    )
    snman.enrichment.match_hierarchy_edits(
        G, hierarchy_edits,
        max_dist=200, max_dist_init=500, max_lattice_width=5,
    )

In [ ]:
from snman.rebuilding import *

HLAs = multi_rebuild_regions(
    G, rebuilding_regions_gdf, access_needs,
    save_steps_path=os.path.join(export_path, 'steps'),
    save_steps_scaling_factor=4,
    gravity_iterations=10,
)
HLAs = list(filter(lambda HLA: HLA is not None, HLAs))

In [ ]:
import networkx as nx

Hs,Ls,As = zip(*HLAs)
A = nx.compose_all(As)
L = nx.compose_all(Ls)

snman.io.export_HLA(os.path.join(export_path, 'steps'), '_debug', L=L, A=A)

In [ ]:
print('Reorder lanes')
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

print('Normalize cycling infrastructure')
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION)
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION_AFTER)

snman.street_graph.organize_edge_directions(G)

In [ ]:
print('Add elevation')
G = oxc.elevation.add_node_elevations_raster(
    G,
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'ch_dhm_25', 'ch_dhm_2056.tif'),
    raster_crs=2056,
    graph_crs=CRS_internal,
    cpus=1
)
G = oxc.elevation.add_edge_grades(G, add_absolute=False)

In [ ]:
snman.space_allocation.generate_lane_stats(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.generate_lane_stats(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

In [ ]:
if 1:
    print('export G')
    snman.io.export_street_graph(
        G,
        os.path.join(export_path, 'G_edges.gpkg'),
        os.path.join(export_path, 'G_nodes.gpkg')
    )

if 1:
    print('export L before')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_nodes.gpkg'),
        scaling=4
    )

if 1:
    print('export L after')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_rebuilt_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_rebuilt_nodes.gpkg'),
        scaling=4
    )